In [10]:
# Importing the required notebooks
%run DifferentialEvolution.ipynb
%run IslandMethod.ipynb

In [11]:
import os
import sys
import numpy as np
import math
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row

from bbobbenchmarks import fgeneric
from bbobbenchmarks import bbobbenchmarks as bn

# Start spark
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('IPDE') \
    .getOrCreate()
sc = spark.sparkContext
    
class Evaluation:

    # Setting the different parameters to evaluate
    _test = {'CR' : '_testCR', 'F' : '_testF'}
    
    
    def __init__(self, title):
        # Setting the CSV title
        self.title = title
        
        # Deleting file if it already exists
        self.filename = "outputs/" + title + ".csv"
        if(os.path.isfile(self.filename)):
            os.remove(self.filename)
            
        # Setting header for the CSV
        output_header = "Function,Instance,Dimensions,NP,CR,F,MuSt,CRSt,Local Iterations,"
        output_header += "Workers,Evaluations,Migrations,Global Time,Chromosome,Target,"
        output_header += "Score,Stop Reason,Restarts"
        self.print_output(output_header)
        
        
    def run(self, test = 'CR', workers = 4, localIt = 2, npop = 80):
        # Setting the functions to test, dimension of individuals and its instan ces
        dimensions = [4]
        function_ids = [3, 6, 10, 15, 16, 20, 23]
        instances = range(1, 4)
        n_restarts = 5

        # Running all functions
        f = fgeneric.LoggingFunction('tmp')
        values = np.arange(0.05, 1.05, 0.05)
        for value in values:
            for dim in dimensions:
                for fun_id in function_ids:
                    for iinstance in instances:
                        # Setting new function to evaluate
                        f.setfun(*bn.instantiate(fun_id, iinstance=iinstance))
                        print("\nFunction:", fun_id, "Instance:", iinstance)
                        
                        # Evaluation of the functions
                        maxEval = self._getMaxEval(fun_id, dim)
                        diffEvoTest = getattr(self, self._test[test])
                        diffEvo = diffEvoTest(f.evalfun, goal = f.ftarget, dim = dim, 
                                              np = npop, maxEval = maxEval, value = value)
                        island = IslandMethod(diffEvo = diffEvo, workers = workers, localIt = localIt)
                        
                        # Run and check if restart is needed
                        restart = -1
                        evaluations = 0
                        while island.stoppingReason != 1 and restart < n_restarts:
                            island.reset_counts()
                            evaluations = island.execute(sc)
                            restart += 1

                        # Storing the results obtained
                        elements = [fun_id, iinstance, dim, diffEvo.np, diffEvo.cr, diffEvo.f,
                                    diffEvo.MuSt, diffEvo.CRSt, localIt, workers, evaluations, 
                                    island.migrationCount, island.globalTime, island.get_best_chromosome(), 
                                    f.ftarget, island.get_best_score(), island.stoppingReason, restart]
                        output_data = ",".join(str(e) for e in elements)

                        # Printing output
                        self.print_output(output_data)
    
    
    def _getMaxEval(self, fun_id, dim):
        maxEval = 5000 * dim**2 
        return maxEval if fun_id == 1 else maxEval * math.log(fun_id)
        

    def _testCR(self, func, goal, dim, np, maxEval, value):
        return DifferentialEvolution(func, goal = goal, np = np, maxEval = maxEval, d = dim, cr = value)
    
    
    def _testF(self, func, goal, dim, np, maxEval, value):
        return DifferentialEvolution(func, goal = goal, np = np, maxEval = maxEval, d = dim, f = value)
    
        
    def print_output(self, output):
        print(output, file = open(self.filename, "a"))

In [ ]:
test = Evaluation("F-test-1")
test.run(test = 'F', workers = 8, localIt = 4, npop = 160)

[Errno 17] File exists: 'tmp'

Function: 3 Instance: 1

Number of Evaluations 160

Initial partition structure: [[(0, (array([ 0.56582621, -2.2482798 ,  3.89065516,  4.52636034]), 38.682241691363686)), (1, (array([-1.21124524,  2.22255461, -0.6290531 , -4.4733627 ]), -157.99733820125635)), (2, (array([-2.70728254, -1.00177627, -2.12123865, -1.64518406]), -245.21100447654837)), (3, (array([ 0.89026201, -2.90210359,  2.44364823,  3.31436479]), -209.47338289105349)), (4, (array([-0.2058342 ,  1.9566886 , -4.32100696,  3.59026312]), -75.580957056924603)), (5, (array([-3.45683004,  2.07079231,  2.08092176, -1.58465542]), -372.39405142264718)), (6, (array([ 0.96653333,  1.10614551, -0.19598534,  4.39431117]), -47.823284902840214)), (7, (array([-1.48157114,  3.91161878,  3.23318115,  0.54949922]), -404.08900394650368)), (8, (array([-3.67781395,  0.65940321,  0.81609981, -4.51479265]), -130.58980467698967)), (9, (array([ 4.49358109, -3.50633833,  1.30272924, -2.20784778]), -251.56587869394434)


Best chromosome ->: [(18, (array([-2.4538947 ,  3.04517914,  2.63921216,  0.97754116]), -436.61433486400591))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.4618999557569623e-05s

Best chromosome ->: [(143, (array([-3.40321625,  2.26647823,  3.16800201,  0.70501258]), -453.59628704846511))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.7667998690740205e-05s

Best chromosome ->: [(85, (array([-3.32697508,  1.5133545 ,  1.79991101,  1.09801072]), -455.90149124367514))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.5204999726847745e-05s

Best chromosome ->: [(115, (array([-1.32455517,  2.94116626,  2.22502128,  0.73722518]), -459.57017140124827))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.097799940907862e-05s

Best chromosome ->: [(28, (array([-2.32079075,  1.5677866 ,  2.2194


Best chromosome ->: [(1, (array([ 4.35430373, -1.58608135, -0.77508909,  3.73557466]), 87.606467913098157))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.0833000487764366e-05s

Best chromosome ->: [(84, (array([ 1.38818543, -0.16138978, -1.69107575,  3.30885108]), 82.05653558837227))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.1101001291535795e-05s

Best chromosome ->: [(66, (array([ 1.38818543, -0.16138978, -1.69107575,  3.30885108]), 82.05653558837227))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.3861999832442962e-05s

Best chromosome ->: [(66, (array([ 1.442179  , -0.89253078, -1.72648048,  3.31665215]), 80.997146828094955))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.0464000297361054e-05s

Best chromosome ->: [(83, (array([ 2.39273352, -0.87302197, -0.77178385,  3


Best chromosome ->: [(140, (array([-0.05419717,  0.94876275,  0.25859833,  3.06588581]), 137.75866698532099))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.1459000234026462e-05s

Best chromosome ->: [(64, (array([-1.16828343, -0.1041456 ,  0.72968894,  2.38932867]), 128.48268696672099))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 2.6787000024341978e-05s

Best chromosome ->: [(93, (array([-1.16828343, -1.7125225 ,  0.72968894,  3.1052228 ]), 119.44032563046751))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.0200001270277426e-05s

Best chromosome ->: [(68, (array([-2.28075686, -1.72142543,  0.24235925,  3.09389298]), 118.61077057977258))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.41910004458623e-05s

Best chromosome ->: [(66, (array([-2.24872181, -0.97446723,  0.23771501, 


Best chromosome ->: [(57, (array([ 1.79478996,  0.6093013 ,  1.15784365,  2.98371392]), 48.043701091854643))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.0816000212798826e-05s

Best chromosome ->: [(104, (array([ 1.79478996,  0.6093013 ,  1.15784365,  2.98371392]), 48.043701091854643))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.162100124929566e-05s

Best chromosome ->: [(11, (array([ 1.24251492,  0.76306592,  2.05604614,  4.31498355]), 45.916284871075455))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.5487999917240813e-05s

Best chromosome ->: [(72, (array([ 1.81738731, -0.05622876,  2.06007524,  3.04038294]), 38.679174537204375))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.0492000001249835e-05s

Best chromosome ->: [(112, (array([ 1.81738731, -0.05622876,  2.06007524

...


Best chromosome ->: [(68, (array([-0.61266816,  2.33419789, -4.81078641,  1.44839732]), -11.13941805373122))]

Migration number: 63

Number of Evaluations 41280

The algorithm stopped because maximum number of evaluations ( 36841.36148790473 ) is reached:  41280

Best chromosome: (-0.612668161336;2.33419788977;-4.81078641377;1.44839732073) Score: -11.1394180537

Global execution time: 0.7170070419997501s

Function: 3 Instance: 1

Number of Evaluations 160

Initial partition structure: [[(0, (array([-0.67640561, -0.3022086 , -2.79560329, -1.02842775]), -243.41083332252924)), (1, (array([ 0.74189303,  2.81610389, -0.07393708,  0.89314779]), -394.21353212799181)), (2, (array([ 4.75056234,  4.70649345,  0.50266458,  3.33741933]), -207.50014595530325)), (3, (array([-3.05744699, -2.44504388, -3.14996766, -2.58455926]), -143.13836746643614)), (4, (array([-3.73489561, -1.38482562, -3.85817009, -1.02006316]), -181.7769571845879)), (5, (array([-2.46960509, -1.81639962,  1.25114387, -3.74742032]


Best chromosome ->: [(32, (array([-2.22362534,  3.04027362,  1.63267879,  1.4361915 ]), -444.52613148039211))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.1910000466741621e-05s

Best chromosome ->: [(20, (array([-3.31609438,  0.25943609,  2.15359567,  1.13606715]), -446.45042810550649))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.5888999769231305e-05s

Best chromosome ->: [(139, (array([-1.34486288,  2.27229334,  2.71953777,  0.68934088]), -456.19696485749273))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.5888999769231305e-05s

Best chromosome ->: [(94, (array([-2.2695579 ,  1.59362865,  2.72516623,  0.40788529]), -456.81197975182022))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 2.915900040534325e-05s

Best chromosome ->: [(68, (array([-2.3293071 ,  2.27541145,  2.18968


Best chromosome ->: [(120, (array([-0.57072009, -0.19003872, -1.80828589,  3.90567537]), 100.23120807208427))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.420800072082784e-05s

Best chromosome ->: [(134, (array([ 0.29364754, -0.20833224, -1.72735246,  3.27345416]), 90.636353531342962))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.515199983259663e-05s

Best chromosome ->: [(99, (array([ 2.48594305, -0.20833224, -1.27167286,  3.27345416]), 84.963675447751285))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.3932000001659617e-05s

Best chromosome ->: [(113, (array([ 2.5375521 , -0.86793157, -1.28215044,  2.9944712 ]), 83.041952674196722))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 2.2869000531500205e-05s

Best chromosome ->: [(101, (array([ 3.3472247 , -0.16525806, -1.2824273


Best chromosome ->: [(81, (array([ 0.99709545,  0.99421542,  0.26984447,  3.50055543]), 139.91232225690882))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.032900036079809e-05s

Best chromosome ->: [(62, (array([-1.22852077, -0.7987319 ,  0.25477922,  3.02704377]), 134.97739325085755))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.0372001270297915e-05s

Best chromosome ->: [(101, (array([-1.19590847, -0.94072868,  0.21043793,  3.7906008 ]), 124.93386550925024))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.7114000002038665e-05s

Best chromosome ->: [(117, (array([-2.08648945, -0.38130158, -0.19250764,  3.47129972]), 121.53861756827141))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.367799995932728e-05s

Best chromosome ->: [(121, (array([-2.14795021, -0.26940157,  0.21856726


Best chromosome ->: [(149, (array([ 0.89179798,  1.11498868,  2.23182153,  3.85207345]), 47.324769565989847))]

Migration number: 1

Number of Evaluations 800
Number of partitions: 8

Island iteration time: 1.1276999430265278e-05s

Best chromosome ->: [(57, (array([ 1.16990247,  1.16718916,  2.30241796,  3.76458482]), 43.426590884051542))]

Migration number: 2

Number of Evaluations 1440
Number of partitions: 8

Island iteration time: 1.3466999007505365e-05s

Best chromosome ->: [(7, (array([ 1.32606434,  1.18414604,  2.70335642,  3.74368029]), 41.747069017623737))]

Migration number: 3

Number of Evaluations 2080
Number of partitions: 8

Island iteration time: 1.4504999853670597e-05s

Best chromosome ->: [(121, (array([ 1.32606434,  1.18414604,  2.70335642,  3.74368029]), 41.747069017623737))]

Migration number: 4

Number of Evaluations 2720
Number of partitions: 8

Island iteration time: 1.5008999980636872e-05s

Best chromosome ->: [(140, (array([ 1.68000768,  0.85086135,  2.3255342

In [ ]:
test = Evaluation("F-test-3_1-local150-max5000")
test.run(test = 'F', workers = 4, localIt = 150, npop = 80)

[Errno 17] File exists: 'tmp'

Function: 3 Instance: 1
Number of partitions: 4

Island iteration time: 9.653000233811326e-06s

Best chromosome ->: [(2, (array([-2.34077855,  2.29852035,  2.21360495,  0.41812407]), -461.09410666913277))]

Migration number: 1

Number of Evaluations 12080
Number of partitions: 4

Island iteration time: 1.0256000223307637e-05s

Best chromosome ->: [(13, (array([-2.34082381,  2.29999918,  2.21360401,  0.72800306]), -462.08999987203526))]

Migration number: 2

Number of Evaluations 24080

The algorithm stopped because the minimum was reached

Best chromosome: (-2.34082381049;2.29999917602;2.21360400741;0.728003056672) Score: -462.089999872

Global execution time: 2.6299216700003853s

Function: 3 Instance: 2
Number of partitions: 4

Island iteration time: 1.0520000159885967e-05s

Best chromosome ->: [(7, (array([ 2.39885856, -0.85322792, -1.27202982,  3.01333995]), 78.65777984753241))]

Migration number: 1

Number of Evaluations 12080
Number of partitions: 4



Best chromosome ->: [(67, (array([-2.18090851, -0.32553073,  0.24479791,  3.08792796]), 116.67748888767242))]

Migration number: 2

Number of Evaluations 24080
Number of partitions: 4

Island iteration time: 1.5375000202766387e-05s

Best chromosome ->: [(14, (array([-2.17840023, -0.32562511,  0.2448    ,  3.08880091]), 116.6749590586447))]

Migration number: 3

Number of Evaluations 36080
Number of partitions: 4

Island iteration time: 1.057800000126008e-05s

Best chromosome ->: [(32, (array([-2.17840001, -0.32562429,  0.24480001,  3.08880001]), 116.6749590570938))]

Migration number: 4

Number of Evaluations 48080
Number of partitions: 4

Island iteration time: 1.541500023449771e-05s

Best chromosome ->: [(6, (array([-2.1784    , -0.32562432,  0.2448    ,  3.0888    ]), 116.6749590570933))]

Migration number: 5

Number of Evaluations 60080
Number of partitions: 4

Island iteration time: 1.0484999620530289e-05s

Best chromosome ->: [(68, (array([-2.1784    , -0.32562432,  0.2448    , 


Best chromosome ->: [(75, (array([ 0.55209334, -0.34561895,  2.67769205,  3.07094723]), 43.233887296416178))]

Migration number: 12

Number of Evaluations 144080

The algorithm stopped because maximum number of evaluations ( 143340.7575382444 ) is reached:  144080

Best chromosome: (0.552093342036;-0.345618950992;2.67769205323;3.07094723099) Score: 43.2338872964

Global execution time: 25.072751147999952s
Number of partitions: 4

Island iteration time: 1.0262000159855234e-05s

Best chromosome ->: [(52, (array([ 1.33910277, -0.78163998,  1.18378467,  3.20921751]), 43.706797773121536))]

Migration number: 1

Number of Evaluations 12080
Number of partitions: 4

Island iteration time: 1.2058999800501624e-05s

Best chromosome ->: [(1, (array([ 1.52008552, -0.72930811,  2.0624319 ,  2.95884056]), 40.502835847664521))]

Migration number: 2

Number of Evaluations 24080
Number of partitions: 4

Island iteration time: 9.63200000114739e-06s

Best chromosome ->: [(27, (array([ 1.53132593, -0.7261